In [9]:
import sqlite3
def get_row_info(fields,table_name,):
        conn = sqlite3.connect('digikala_database-old.db',timeout=30)
        cursor = conn.cursor()
       
        field = ','.join(fields)
        query = f"SELECT {field} FROM {table_name} "
        print(query)
        cursor.execute(query)
        row_info = cursor.fetchall()
        conn.close()
        return  [product[0] for product in row_info]

get_row_info(fields=['seller_id'],table_name='sellers')

SELECT seller_id FROM sellers 


['5A52N',
 '5AD97',
 '9PAT4',
 'A4F6F',
 'A4ZKW',
 'A97EK',
 'AA9FR',
 'AATCE',
 'ACMWJ',
 'AN4UE',
 'AUFF9',
 'AUSNV',
 'C9YXK',
 'C9Z34',
 'CE3HE',
 'CGDG9',
 'CGXGH',
 'CHG4X',
 'CKKCP',
 'CSZ49',
 'CYGAV',
 'D7ZNU',
 'D9AC2',
 'D9E6E',
 'DGWEA',
 'DPMUV',
 'DRCZ2',
 'DU3ZS',
 'DVEFT',
 'E5SWN',
 'E7PAC',
 'E7Z6Z',
 'EDV9Y',
 'EE6DA',
 'EJM9M',
 'EKV5H',
 'EM6DP',
 'EPMCW',
 'EW2A5',
 'F7TMP',
 'FJC6P',
 'FM9XU',
 'FNECY',
 'egjaf']

In [3]:
import re
def get_url_input( mode):
        while True:
            fmode = mode.replace('_',' ')
            url = input(f'Enter {fmode} URL: ')

            if mode == 'single_product':
                if ('product' not in url and not re.search(r'dkp-\d+', url) ):
                    print('URL is not belong to any product check it and Please try again.')
                    continue

            elif mode == 'single_seller':
                pattern = re.compile(r'https://www\.digikala\.com/seller/[A-Z0-9]+/$')
                if  ('seller' not in url and not pattern.match(url)):
                     print("URL is not belong to any seller check it and Please try again.")
                     continue
            elif mode == 'category':
                if not ('search/?q=' in url or '/category-' in url or '/search/' in url):
                    print('Wrong category URL, try one more time')
                    continue

            break
        return url
get_url_input( 'single_seller')

'https://www.digikala.com/seller/es4mf/asd'

In [ ]:
def run_scraper_category(self):
    pass

In [ ]:
def run_scraper_single(self):
    pass


In [ ]:
def run_seller_scraper_product(self):
        row_info = self.db_handler.get_row_info(fields=['seller_id', 'seller_name'], table_name='sellers',condition=None,return_as_list=False)
        if len(row_info) == 0 :
            self.log.warning("There are no sellers in the database.")
            return None
        
        for index, row in enumerate(row_info):
            self.log.info(f"ID {index} : {row[0]}, Name: {row[1]}")
        while True:
            try:
                user_pick = int(input('choose the seller ID you want to crawl products from: '))
                if 0 <= user_pick < len(row_info):
                    break
                else:
                    self.log.info('Invalid choose, please try again.')
            except ValueError:
                self.log.info('Invalid input, please enter a number.')
        selected_seller = row_info[user_pick]
        self.get_driver()
        self.log.info(f'You chose {selected_seller[1]} with ID {selected_seller[0]}')    
        seller_products = self.db_handler.get_row_info(['product_link', 'product_price'], 'products', ['seller_name', selected_seller[1]])
        available_products = [product[0] for product in seller_products if product[1] != 'product unavailable']
        self.log.info(f'{len(available_products)} available products found for {selected_seller[1]}')
        for index, link in enumerate(available_products):
            self.log.info(f'Starting to crawl - {index + 1}/{len(available_products)}')
            self.product_extraction_scraper.run(link)


In [ ]:
def run_single_scraper_product(self):
    pass



In [ ]:
def scraper_all_product_on_db(self,):
    pass
